In [1]:
import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
import tensorflow.keras.backend as K
from commai_spd_chl_keras_datagen import commai_speed_chl_gen
from tensorflow.keras.layers import Dense,Activation,Lambda,GRU,BatchNormalization,Activation,Flatten,Conv2D,MaxPooling2D,Reshape,Dropout

Using TensorFlow backend.


In [2]:
tf.config.list_physical_devices ()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.keras.backend.image_data_format()

'channels_last'

In [4]:
seq = Sequential()

#should I trust this guy?https://www.kdnuggets.com/2018/09/dropout-convolutional-networks.html

seq.add(Lambda(lambda x:tf.divide(x,255),input_shape=(480*2,640,1))) #scale the inputs between 0-1

seq.add(Conv2D(filters=32, kernel_size=(4,4),padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=32, kernel_size=(4,4),padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Conv2D(filters=64, kernel_size=(4,4),padding='same'))
seq.add(MaxPooling2D(pool_size=(4,4)))
seq.add(Activation("relu"))
seq.add(BatchNormalization())

seq.add(Flatten())

seq.add(Dense(192,))
seq.add(Activation("relu"))
seq.add(BatchNormalization())
seq.add(Dropout(.3)) # throw away 30%

seq.add(Dense(50,))
seq.add(Activation("relu"))
seq.add(Dropout(.2))

seq.add(Dense((2)))

seq.add(Reshape((2,1))) #this sucks dick(s)

seq.compile(loss='mse', optimizer='adam')
seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 960, 640, 1)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 960, 640, 32)      544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 240, 160, 32)      0         
_________________________________________________________________
activation (Activation)      (None, 240, 160, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 240, 160, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 240, 160, 32)      16416     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 40, 32)        0

In [5]:
batch_size=32
n_e=60
n_frames_train=15000
n_frames_test=2900
n_frames_valid=2500

train_idx,test_idx,valid_idx=commai_speed_chl_gen(n_frames_valid=n_frames_valid,frames_per_sample=2,mode="stacked",
                    n_frames_test=n_frames_test,n_frames_train=n_frames_train,batch_size=batch_size).get_indexes()

#load=np.load(open("idxes_21bs_STACKED_15k.np","rb"))
##train_idx=load["train_idx"]
#test_idx=load["test_idx"]
#valid_idx=load["valid_idx"]

train_gen=commai_speed_chl_gen(frame_idx=train_idx,batch_size=batch_size,mode="stacked",frames_per_sample=2)
test_gen=commai_speed_chl_gen(frame_idx=test_idx,batch_size=batch_size,mode="stacked",frames_per_sample=2)
valid_gen=commai_speed_chl_gen(frame_idx=valid_idx,batch_size=batch_size,mode="stacked",frames_per_sample=2)

np.savez_compressed(open("idxes_21bs_STACKED_15k.np","wb+"),train_idx=train_idx,test_idx=test_idx,valid_idx=valid_idx)



(480, 640, 3)
(32, 960, 640)
(32, 2, 1)
train_samples: 7500 
test_samples:1450 
valid_samples: 1250
(480, 640, 3)
(32, 960, 640)
(32, 2, 1)
(480, 640, 3)
(32, 960, 640)
(32, 2, 1)
(480, 640, 3)
(32, 960, 640)
(32, 2, 1)


In [6]:
ie=0
filepath="STACKED_CNN_2FPS_Block_approach_32bs-{epoch:02d}-{val_loss:.2f}-{loss:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False)

tb=tf.keras.callbacks.TensorBoard(
    log_dir='STACKED_CNN_2FPS_Block_approach_32bs', histogram_freq=1, write_graph=True, write_images=True,
    update_freq=10, profile_batch=0, embeddings_freq=0
)

cbs=[checkpoint,tb]
hst=seq.fit_generator(train_gen, epochs=n_e,workers=30,max_queue_size=100,
        use_multiprocessing=True,validation_data=test_gen,callbacks=cbs,verbose=1,shuffle=True, initial_epoch=ie)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/60
234/234 [==============================] - ETA: 0s - loss: 91.2764WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to STACKED_CNN_2FPS_Block_approach_32bs-01-194.20-91.28.hdf5
234/234 [==============================] - 150s 640ms/step - loss: 91.2764 - val_loss: 194.1988
Epoch 2/60
234/234 [==============================] - ETA: 0s - loss: 53.8915WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to STACKED_CNN_2FPS_Block_approach_32bs-02-105.61-53.89.hdf5
234/234 [==============================] - 152s 649ms/step - loss: 53.8915 - val_loss: 105.6092
Epoch 3/60
234/234 [==============================] - ETA: 0s - loss

Epoch 9/60
234/234 [==============================] - ETA: 0s - loss: 19.0913WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00009: saving model to STACKED_CNN_2FPS_Block_approach_32bs-09-20.80-19.09.hdf5
234/234 [==============================] - 149s 639ms/step - loss: 19.0913 - val_loss: 20.7984
Epoch 10/60
234/234 [==============================] - ETA: 0s - loss: 17.0709WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00010: saving model to STACKED_CNN_2FPS_Block_approach_32bs-10-12.94-17.07.hdf5
234/234 [==============================] - 152s 648ms/step - loss: 17.0709 - val_loss: 12.9441
Epoch 11/60
234/234 [==============================] - ETA: 0s - loss: 15.4330WARNING:tensorflow:multiprocessing can interact badly with TensorFlow

234/234 [==============================] - ETA: 0s - loss: 10.6899WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00018: saving model to STACKED_CNN_2FPS_Block_approach_32bs-18-26.20-10.69.hdf5
234/234 [==============================] - 148s 633ms/step - loss: 10.6899 - val_loss: 26.2035
Epoch 19/60
234/234 [==============================] - ETA: 0s - loss: 10.3414WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00019: saving model to STACKED_CNN_2FPS_Block_approach_32bs-19-17.65-10.34.hdf5
234/234 [==============================] - 154s 660ms/step - loss: 10.3414 - val_loss: 17.6470
Epoch 20/60
234/234 [==============================] - ETA: 0s - loss: 10.1039WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing n

Process Keras_worker_ForkPoolWorker-2456:
Process Keras_worker_ForkPoolWorker-2466:
Process Keras_worker_ForkPoolWorker-2481:
Process Keras_worker_ForkPoolWorker-2453:
Process Keras_worker_ForkPoolWorker-2452:
Process Keras_worker_ForkPoolWorker-2461:
Process Keras_worker_ForkPoolWorker-2435:
Process Keras_worker_ForkPoolWorker-2434:
Process Keras_worker_ForkPoolWorker-2438:
Process Keras_worker_ForkPoolWorker-2432:
Process Keras_worker_ForkPoolWorker-2439:
Process Keras_worker_ForkPoolWorker-2462:
Process Keras_worker_ForkPoolWorker-2444:
Process Keras_worker_ForkPoolWorker-2468:
Process Keras_worker_ForkPoolWorker-2441:
Process Keras_worker_ForkPoolWorker-2485:
Process Keras_worker_ForkPoolWorker-2460:
Process Keras_worker_ForkPoolWorker-2479:
Process Keras_worker_ForkPoolWorker-2449:
Process Keras_worker_ForkPoolWorker-2483:
Process Keras_worker_ForkPoolWorker-2471:
Process Keras_worker_ForkPoolWorker-2431:
Process Keras_worker_ForkPoolWorker-2480:
Process Keras_worker_ForkPoolWorke

  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  Fi

  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))


  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 347, in put
    self._writer.send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._wlock:
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 346, in put
    with self._w

  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py",

KeyboardInterrupt: 

In [ ]:
#15 Epoches
seq=load_model("STACKED_CNN_2FPS_Block_approach_32bs-15-10.02-12.41.hdf5")
seq.evaluate_generator(valid_gen,n_frames_valid/batch_size/2, max_queue_size = 80,workers = 10,verbose=2)

Instructions for updating:
Please use Model.evaluate, which supports generators.
